In [27]:
import numpy as np
import pandas as pd
import time

## ANTICOR

Can we learn to beat the best stock. JAIR, 21:579–594, 2004.

Reference:
Borodin, A., El-Yaniv, R., and Gogan, V. 
http://www.cs.toronto.edu/~bor/Papers/beating-best-stock.pdf

In [28]:
data=pd.read_csv('./Data_OLPS/nyse-n.csv',encoding="gb18030")

print('data.shape：',data.shape) 

#data.info()
N=data.shape[0]
d=data.shape[1]
x=np.zeros((N,d))
b=np.ones(d)/d
x=data.to_numpy()
p=x
x

data.shape： (6431, 23)


array([[0.99751   , 0.98648   , 0.9922    , ..., 0.98684   , 0.99671   ,
        1.00892   ],
       [1.00994   , 1.01371   , 1.00198   , ..., 0.99333   , 0.95396   ,
        0.99116   ],
       [0.99753   , 0.96957   , 0.98433   , ..., 1.00224   , 0.99654   ,
        1.00448   ],
       ...,
       [0.99315069, 0.98214286, 0.98578199, ..., 1.01349073, 0.98453608,
        0.99439776],
       [0.97931035, 0.93636364, 0.95913461, ..., 0.99500832, 0.95986038,
        0.9915493 ],
       [1.        , 0.97087379, 0.97994987, ..., 0.99498328, 1.00545455,
        0.98153409]])

In [29]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m

    return np.maximum(y-tmax,0.)


### Algorithm 1. Portfolio Selection with OLMAR.
1. Input: $𝜖>1$: Reversion threshold; $w>3$: Window size; $x_1^n$: Market sequence.
2. Output: $S_n$: Cumulative wealth after $n^{th}$ periods.
3. Procedure:
4. Initialize $𝒃_𝟏 =(\frac{1}{𝑚},⋯,\frac{1}{𝑚}),S_0 = 1$ 
5. for t = 1, 2,...,n do
6. $\quad$Receive stock price relatives: $𝑥_𝑡=(𝑥_{t1},⋯,𝑥_{tm})$
7. $\quad$Calculate daily return and cumulative return: $S_t=S_{t-1}×(b_t·x_𝑡)$
8. $\quad$Predict next price relative vector: 
$$
x̃_{𝑡+1}= \frac{1}{w} (1+\frac{1}{x_𝑡}+ ⋯ + \frac{1}{⊗_{i=0}^{w-2}\quad x_{t-i}})
$$

9. $\quad$Update the portfolio:

$$
b_{t+1}=OLMAR(ϵ, w, x_{t+1}, b_t)
$$

### Algorithm   ANTICOR($w, t, x_t, \hat b_t$)
Input:

   1. $w$: Window size; 
   2. $t$: Index of last trading day; 
   3. $X_𝑡=(x_1,⋯,x_t)$: Historical market sequence;
   4. $\hat b_t$:  current portfolio (by the end of trading day t).
   
Output: $b_{𝑡+1}$:Next day’s portfolio
 
   1. Return the current portfolio $\hat b_t$ if $t<2w$.
   2. Compute $LX_1$ and $LX_2$ as defined in Equation (2), and $µ_1$ and $µ_2$, the (vector) averages of $LX_1$ and $LX_2$, respectively.
   3. Compute $M_{cor}(i,j)$ as defined in Equation (3).
   4. Calculate claims: for $1≤i,j≤m$, initialize $claim_{i→j}=0$
   5. If $µ_2(i)≥µ_2(j)$ and $M_{cor}(i,j)>0$ then
   
       (a) $claim_{i→j}=$claim_{i→j}+M_{cor}(i,j)$;
       
       (b) if $M_{cor}(i,i)<0$ then $claim_{i→j}=claim_{i→j}-M_{cor}(i,i)$;
       
       (c) if $M_{cor}(j,j)<0$ then $claim_{i→j}=claim_{i→j}-M_{cor}(j,j)$;
       
       
   6. Calculate new portfolio: Initialize $b^{t+1} = \hat bt$. For $1≤i,j≤m$
   
       (a) Let $transfer_{i→j} = b^t_i · claim_{i→j}/\sum_{j}claim_{i→j}$;
       
       (b) $b^{t+1}_i=b^{t+1}_i-transfer_{i→j}$;
       
       (c) $b^{t+1}_i=b^{t+1}_i+transfer_{i→j}$; 

In [30]:
start = time.time()
cum_ret=1 # Initialization
w=5 # Window size
# w=np.arange(5,100,5)
#Inputs: w = window size, t = index of last trading day, x = Historical market sequence of returns (p / p_[t-1]), b_hat = current portfolio (at end of trading day t).  Initialize m to width of matrix x (= number of stocks)
b=1. / d * np.ones(data.shape)
daily_r=np.ones(N)
for t in range(N-1):
    # 1. Return the current portfolio ˆbt if t < 2w
    if t < 2*w - 1:
        b
    else:
    # 2.1 Compute LX1, LX2
        LX1 = np.log(x[t-2*w+1:t-w+1,:])
        LX2 = np.log(x[t-w+1:t+1,:])
    # 2.2 Compute averages of LX1 and LX2
        mu1 = np.mean(LX1, axis=0) 
        mu2 = np.mean(LX2, axis=0)
    # 3 Compute Mcor(i, j)
        M_cov = np.zeros((d,d))
        M_cor = np.zeros((d,d))
        claim = np.zeros((d,d))
        transfer = np.zeros((d,d))        
        sig1 = np.std(LX1, axis=0)
        sig2 = np.std(LX2, axis=0)
        sigma = sig1@sig2
        
    # 4-5
        for i in range(d):
            for j in range(d):
                M_cov[i,j] = np.dot(np.transpose(LX1[:,i]-mu1[i]),(LX2[:,j] - mu2[j]))/(w-1)
                # 3
                if (sigma == 0):
                    M_cor[i,j] = 0
                else:
                    M_cor[i,j] = M_cov[i,j] / sigma
                    
                if(i!=j):
                    if ((mu2[i] >= mu2[j]) & (M_cor[i,j] > 0)):
                        claim[i,j] = claim[i,j] + M_cor[i,j]
                        if (M_cor[i,i] < 0):
                            claim[i,j] = claim[i,j] + np.maximum(0, -M_cor[i,i])
                        if (M_cor[j,j] < 0):
                            claim[i,j] = claim[i,j] + np.maximum(0, -M_cor[j,j])
            
    # 6   
        for i in range(d):   
            total_claim = 0
            for j in range(d):
                total_claim += claim[i,j]
            if(total_claim != 0):
                for j in range(d):
                    transfer[i,j] = b[t,i] * claim[i,j] / total_claim
        
        for i in range(d): 
            b[t+1,i] = b[t,i]
            for j in range(d):
                # update weights    
                b[t+1,i] +=transfer[j,i] - transfer[i,j]

        b[t+1] = simplex_proj(b[t+1])
#     tc = gamma/2 * (np.abs(((x[t+1]*b)/ (np.dot(x[t+1],b)))[1:] - b[1:])).sum()
        daily_r[t] = np.dot(x[t+1],b[t+1]) #* (1-tc)
        cum_ret = cum_ret * daily_r[t] 
print("Cumulative return:",cum_ret,"-----> Expressed by scientific counting:",f'{cum_ret:1.2e}') 

end = time.time()
print("Program running time:",end-start)

5
Cumulative return: 2456685.767054648 -----> Expressed by scientific counting: 2.46e+06
Program running time: 72.14797377586365


In [31]:
daily_r = pd.DataFrame(daily_r,index = data.index,columns=['daily return'])
# print(daily_r)
mean_daily = daily_r['daily return'].mean()
print("Average daily rate of return:",mean_daily)
mean_return_annualized = mean_daily**252 - 1
# print("mean_return_annualized:",mean_return_annualized)

std_daily = np.std(np.array(daily_r))
variance_daily = std_daily ** 2
std_annualized = std_daily*np.sqrt(252)
variance_annualized = std_annualized ** 2
volatility = std_daily * np.sqrt(252)
print("Volatility is: ",volatility)
SR = mean_daily/std_daily
print("Daily Sharpe ratio is: ",SR)
ASR=np.sqrt(252)*SR
maximum = np.maximum.accumulate(np.array(daily_r))
MDD = ((maximum - np.array(daily_r)) / maximum).max()
print("Max Drawdown is: ",MDD)
CS = mean_return_annualized/MDD
print("Calmar Ratio is: ",CS)

Average daily rate of return: 1.0027230957751705
Volatility is:  0.46806471868115423
Daily Sharpe ratio is:  34.007552879743066
Max Drawdown is:  0.59962571386484
Calmar Ratio is:  1.6415944847299775
